In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import cv2
import numpy as np
from PIL import Image, ImageDraw
import tensorflow as tf
import json
import os
from tqdm import tqdm

import sys
sys.path.append('/home/dan/work/cocoapi/PythonAPI/')

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import matplotlib.pyplot as plt
%matplotlib inline

from detector import Detector

In [ ]:
MODEL_PATH = 'model.pb'
detector = Detector(MODEL_PATH)

# Load COCO groundtruth

In [ ]:
IMAGES_DIR = '/mnt/datasets/COCO/images/'
ANNOTATIONS_DIR = '/mnt/datasets/COCO/annotations/'
DATA_TYPE = 'val2017'  # train2017 or val2017

In [ ]:
coco = COCO(os.path.join(ANNOTATIONS_DIR, 'instances_{}.json'.format(DATA_TYPE)))

# Load labels

In [ ]:
with open('../data/coco_labels.txt', 'r') as f:
    labels = {line.strip(): i for i, line in enumerate(f.readlines()) if line.strip()}

In [ ]:
categories = coco.loadCats(coco.getCatIds())
class_to_coco_id = {c['name']: c['id']  for c in categories}

# my integer encoding (from 0 to 79) to official integer encoding
integer_to_coco_id = {labels[n]: i for n, i in class_to_coco_id.items()}

# Compute predictions

In [ ]:
catIds = coco.getCatIds()
imgIds = coco.getImgIds()
len(imgIds)

In [ ]:
results = []

for image_id in tqdm(imgIds):

    image_metadata = coco.loadImgs(image_id)[0]
    image_path = os.path.join(IMAGES_DIR, DATA_TYPE, image_metadata['file_name'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = image.shape

    boxes, labels, scores = detector(image, score_threshold=0.15)

    for i in range(len(boxes)):
        ymin, xmin, ymax, xmax = boxes[i]

        x, y = int(xmin), int(ymin)
        w, h = int(xmax - xmin), int(ymax - ymin)

        results.append({
            "image_id" : int(image_metadata['id']), "category_id" : int(integer_to_coco_id[labels[i]]), 
            "bbox" : [x, y, w, h], "score" : float(scores[i])
        })

In [ ]:
with open('coco_predictions.json', 'w') as f:
    json.dump(results, f)

# Show detections

In [ ]:
coco_detections = coco.loadRes('coco_predictions.json')

In [ ]:
def get_random():
    
    i = np.random.randint(0, len(imgIds))
    image_id = imgIds[i]
    image_metadata = coco.loadImgs(image_id)[0]
    
    image_path = os.path.join(IMAGES_DIR, DATA_TYPE, image_metadata['file_name'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    annIds = coco_detections.getAnnIds(imgIds=image_metadata['id'], catIds=catIds)
    annotations = coco_detections.loadAnns(annIds)
    return image, annotations

In [ ]:
image, annotations = get_random()

plt.figure(dpi=100, figsize=(17, 10))
plt.imshow(image)
plt.axis('off')
coco.showAnns(annotations);

# Evaluate

In [ ]:
cocoEval = COCOeval(cocoGt=coco, cocoDt=coco_detections, iouType='bbox')
cocoEval.params.imgIds = imgIds
cocoEval.params.catIds = catIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()